# NavigatorGPT - MongoDB Processing

In [170]:
import json
import pymongo
from pymongo import MongoClient, UpdateOne
from bson.objectid import ObjectId

# Load the MongoDB connection string from the JSON file
PATH_TO_SECRET_JSON = '/home/jovyan/keys/mongodb_key.json'
with open(PATH_TO_SECRET_JSON) as f:
    MONGODB_URI = json.load(f)['connection_string']

# Extract the database name from the connection string
db_name = MONGODB_URI.split('/')[-1].split('?')[0]

# Create a MongoDB client using the connection string
if MONGODB_URI:
    client = MongoClient(MONGODB_URI)
    print("The MongoDB client has been initialized.")
else:
    print("Failed to initialize the MongoDB client.")

# Print the version of the pymongo package
pymongo_version = pymongo.__version__
print(f"The version of the pymongo package is {pymongo_version}")

# Connect to the specified MongoDB database and collection
db = client[db_name]
collection = db.attractions

The MongoDB client has been initialized.
The version of the pymongo package is 4.4.1


In [171]:
# Find the document with the specified Objectid in the 'attractions' collection
document = collection.find_one({"_id": ObjectId("6450c55c3879cf70e53ff044")})

# Print the retrieved document
print(document)


{'_id': ObjectId('6450c55c3879cf70e53ff044'), 'title': 'Schermerhorn Building', 'key': 'SchermerhornBuilding', 'featureKey': 'landmark', 'catalog': 'nycwayfinding', 'loc': {'lat': 40.72761, 'lon': -73.99378, 'location': '376-380 Lafayette Street', 'neighborhood': 'Greenwich Village', 'postalCode': '10012', 'city': 'New York', 'state': 'NY', 'boroughCode': 'MN'}, 'map': {'center': {'latitude': 40.72761, 'longitude': -73.99378}, 'zoom': 17}, 'machineTags': [{'tag': 'nycwayfinding:lp=00193'}], 'photo': {'_id': ObjectId('649340baf86d4f4d60e5f631'), 'photoId': 52920417345, 'title': 'Schermerhorn Building', 'url': 'https://live.staticflickr.com/65535/52920417345_6a30663600_t.jpg', 'width': 100, 'height': 75}, 'inventory': {'styles': ['N/A'], 'architects': ['Henry J. Hardenbergh']}, 'aliases': ['376-380 Lafayette Street Building']}


**Description:**
We have a dataset of attractions with specific fields to be updated in the database. Below is the dataset:

In [201]:
data =  [

        {
            "id": "550ddaa8610bf1c45d848997",
            "inventory.materials": "Bronze",
            "inventory.dimensions": "H: 8\" W: 12\"\u00be\""
        },
        {
            "id": "56b6f541cd15882254355aae",
            "inventory.materials": "Bronze",
            "inventory.dimensions": "2'8 \"\u00bd\"h x 2' 8 \"\u00bd\"w"
        },
        {
            "id": "56b5972dcd15881700bb2b21",
            "inventory.materials": "Bronze, granite",
            "inventory.dimensions": "18' 5\" H,"
        },
        {
            "id": "56bf870ecd158804e462eced",
            "inventory.materials": "Bronze plaque on flagpole",
            "inventory.dimensions": "Plaque H: 14\" W: 9\""
        },
        {
            "id": "53668a9a5c959622000890c7",
            "inventory.materials": "Sculpture--cast iron and welded steel (painted)",
            "inventory.dimensions": "H: 13' L: 7'10\" W: 5'6\""
        },
        {
            "id": "56bf982ecd158827a80aef2b",
            "inventory.materials": "Bronze, Maine granite"
        },
        {
            "id": "533cddb35c9596ef08143de7",
            "inventory.materials": "Swensons pink granite (honed finish)",
            "inventory.dimensions": "H: 8'8\"  Diameter: 10'9\""
        },
        {
            "id": "550632b193a2b1e1e52db6c1",
            "inventory.materials": "Lamp--cast iron; plaque--aluminum",
            "inventory.dimensions": "H: 15'; Weight: 1,320 pounds"
        },
        {
            "id": "56c0008bcd15882fe0c9864e",
            "inventory.materials": "Bronze",
            "inventory.dimensions": "H: 27\" W: 38\""
        },
        {
            "id": "56c284126e78364115ed87a1",
            "inventory.materials": "bronze"
        },
        {
            "id": "56aabab5cd158826d842d773",
            "inventory.materials": "Upper portion--Indiana limestone; Base and basin--Stony Creek granite",
            "inventory.dimensions": "H: 7'8\" W: 17'4\" D: 10'3\""
        },
        {
            "id": "55efca10f58614cf075b75d5",
            "inventory.materials": "bronze",
            "inventory.dimensions": "48\"h x 36\"w x \"\u00bd\"d"
        },
        {
            "id": "56c6a32762a3debc2b358e69",
            "inventory.materials": "Concord granite",
            "inventory.dimensions": "H: 7'6\" W: 60\" D: 55\""
        },
        {
            "id": "55fb945af58614cf075b76b4",
            "inventory.materials": "Bronze",
            "inventory.dimensions": "H: 12\" W: 18\""
        },
        {
            "id": "56b2d697cd158828b067f332",
            "inventory.materials": "Bronze, granite",
            "inventory.dimensions": "H: 9\" W: 13\""
        },
        {
            "id": "56b40b98cd158811a84fceec",
            "inventory.materials": "Bronze, polished Quincy granite",
            "inventory.dimensions": "H: 15' W: 6' D:6'"
        },
        {
            "id": "550cf980610bf1c45d848963",
            "inventory.materials": "Colorado black marble and white marble",
            "inventory.dimensions": "H: 5'6\""
        },
        {
            "id": "56b6e8c1cd15882f6492b9e9",
            "inventory.materials": "Bronze",
            "inventory.dimensions": "H: 2' W: 2'9\""
        },
        {
            "id": "56cd249e62a3debc2b358efb",
            "inventory.materials": "Bronze, steel",
            "inventory.dimensions": "see memo"
        },
        {
            "id": "54f7e9f95c95968fe44879a4",
            "inventory.materials": "Granite, bronze, marble",
            "inventory.dimensions": "Column H: 12' Diameter 19\" - 26\"; Capital H: 29\" W: 29\" D: 22 1/2\"; Globe Diameter 24\""
        },
        {
            "id": "56cfce4a62a3debc2b358f3a",
            "inventory.materials": "indiana limestone"
        },
        {
            "id": "533cddb25c9596ef08143dde",
            "inventory.materials": "Bronze and granite"
        },
        {
            "id": "55fd7912f58614cf075b76e0",
            "inventory.materials": "Bronze, Quincy granite",
            "inventory.dimensions": "H: 14'11\" W: 10'4\" D: 10'4\""
        },
        {
            "id": "56c298b76e78364115ed87ba",
            "inventory.materials": "Bronze, natural boulder",
            "inventory.dimensions": "Tablet H: 18\" W: 22\"; Boulder H: 4' W: 3' D: 3'"
        },
        {
            "id": "56bfd34fcd15882f9c38cccd",
            "inventory.materials": "Paint",
            "inventory.dimensions": "Each H: 10'  W: 20'"
        },
        {
            "id": "56d7ab7962a3debc2b358f8d",
            "inventory.materials": "Barre granite, bronze",
            "inventory.dimensions": "Plinth H: 1' W: 6' D: 2'10\"; Stele H: 5'7\"\u00bd\" W: 4' D: 1'6\"; Plaque H: 2'6\" W: 2'9\"\u00bd\""
        },
        {
            "id": "55ee4cf4f58614cf075b75ab",
            "inventory.materials": "Bronze, Stony Creek granite",
            "inventory.dimensions": "20'h x 40\"w x 20\"d"
        },
        {
            "id": "56d68cd762a3debc2b358f87",
            "inventory.materials": "Bronze, concrete",
            "inventory.dimensions": "H: 23\" W: 36 \"\u00bd\" D: \"\u00bd\""
        },
        {
            "id": "5528b75b912913b260eb7605",
            "inventory.materials": "Bronze, Milford pink granite",
            "inventory.dimensions": "Figure h 6'4\" X w 2'5\"; Stele h 10'  X  w 8' X  d 2'7\"; each tablet  h 4'9\" X  w 1'3\""
        },
        {
            "id": "56aabab5cd158826d842d784",
            "inventory.materials": "Chair and dictionary--bronze (light blue-green patina); Boulder--Stony Creek, Connecticut pink granite",
            "inventory.dimensions": "Armchair H: 4'5\" W: 2'5\" D: 2'6\"; Boulder H: 4' W: 6'8\" L: 8'8\""
        },
        {
            "id": "56c277686e78364115ed8790",
            "inventory.materials": "Bronze, wood, granite base",
            "inventory.dimensions": "12'h x 9 \"\u00bc\"w x 13\"diam (pole"
        },
        {
            "id": "55236623610bf1c45d848a5a",
            "inventory.materials": "maine granite"
        },
        {
            "id": "54f7e2b193a2b1e1e52db53d",
            "inventory.materials": "Cast stone",
            "inventory.dimensions": "H: 2'9\" W: 1'11\" D: 1'11\""
        },
        {
            "id": "56bfb26dcd15881fe4bbc059",
            "inventory.materials": "Figure--bronze; Base--black Carolina granite",
            "inventory.dimensions": "Figure H: 6'; Base H: 7'6\" W: 4' D: 7'6\""
        },
        {
            "id": "55fd7b07f58614cf075b76ea"
        },
        {
            "id": "56bf6c4bcd15881494a0f17e",
            "inventory.materials": "Bronze, North Jay granite",
            "inventory.dimensions": "Figure H: 6'7\"; Pedestal H: 6' W: 5'10\" D: 5'10\"; Plaques H: 1'8\" W: 2'6\""
        },
        {
            "id": "55f246f3f58614cf075b75e6",
            "inventory.materials": "Bronze, natural boulder",
            "inventory.dimensions": "Tablet H: 2' W: 1'2\"; Boulder H: 5'8\" W: 4'6\""
        },
        {
            "id": "56bfcc5acd15882db4ad1271",
            "inventory.materials": "Bronze, Stony Creek granite",
            "inventory.dimensions": "Tablet H: 2'2\" W: 3'9\" D: 1'1\"; Base Diameter\" 9'"
        },
        {
            "id": "533cddb35c9596ef08143dfc",
            "inventory.materials": "Concord white granite, bronze",
            "inventory.dimensions": "H: 3'7\" W: 27'3\" D:16'4\""
        },
        {
            "id": "56aabab5cd158826d842d782",
            "inventory.materials": "Milford pink granite",
            "inventory.dimensions": "H: 16' W: 6'6\" D: 6'6\""
        },
        {
            "id": "56b68230cd15882c40f88b19",
            "inventory.materials": "Deer Isle granite",
            "inventory.dimensions": "H: 5'10\" W: 2' D: 2'"
        },
        {
            "id": "56bfad66cd15881e4c9718e7",
            "inventory.materials": "Cor-ten steel, painted red",
            "inventory.dimensions": "H: 4\"\u00bd' W: 4\"\u00bd' L: 12'"
        },
        {
            "id": "55e65b18f58614cf075b74f0",
            "inventory.materials": "Bas-relief--Indiana limestone; Plaque--bronze",
            "inventory.dimensions": "H: 13' W: 20'8\" D: 3'10\""
        },
        {
            "id": "550643e193a2b1e1e52db6cd",
            "inventory.materials": "Bronze",
            "inventory.dimensions": "H: 1'1\" W: 1'4 \"\u00bd\""
        },
        {
            "id": "552db730912913b260eb7695"
        }






 
];

**Description:**
Now, we'll loop through the dataset and update the records in the MongoDB collection.

In [202]:
# Initialize counters and operations list
totalRecords = len(data)
recordsUpdated = 0
operations = []

# Prepare bulk update operations for each record in data
for item in data:
    update_fields = {}

    # Iterate over all keys in the item
    for key, value in item.items():
        # Skip 'id' key
        if key == 'id':
            continue 
        
        # Check if the value is a dictionary (nested dictionary in MongoDB)
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                update_fields[f"{key}.{sub_key}"] = sub_value
        else:
            update_fields[key] = value

    operations.append(
        UpdateOne(
            {"_id": ObjectId(item['id'])},  # corrected here
            {"$set": update_fields},
            upsert=True
        )
    )

# Execute the update operations in bulk
result = collection.bulk_write(operations)

# Calculate the total records updated
recordsUpdated = result.modified_count + len(result.upserted_ids)

# Output the results
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")


Total records: 45
Records updated: 35


**Description:**
Print out the results, showing the total number of records processed and the number of records updated.

In [203]:
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")

Total records: 45
Records updated: 35
